## Tutorial for contextual-AI with H2O Model

This tutorial is to show how to create a wrapper for H2O model and make it compatible with contextual-ai library function calls.

We will train a simple `H2ORandomForestEstimator` model to classify the iris dataset, in the end we want to generate a model interpretation reports including the following:
- Feature importance globally from shap values
- Model per-class feature importance by aggregating explanations

Note: please install h2o library in order to run this tutorial.

In [1]:
import sys
import json

import sklearn
import sklearn.datasets
import sklearn.ensemble
import pandas as pd
import numpy as np
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator

np.random.seed(1)

We will start the demo by training a H2O model with iris data.

### Initialize the H2O virtual cluster


In [2]:
h2o.init(max_mem_size = "500M", nthreads = 6) 

# h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,16 hours 1 min
H2O_cluster_timezone:,Asia/Singapore
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,29 days
H2O_cluster_name:,H2O_from_python_i309943_1ksyg3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.546 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


### Data preparation for training a H2O model

1. Load the dataset and split the data into train and test

In [3]:
iris = sklearn.datasets.load_iris()

feature_names = iris.feature_names
class_labels = 'species'

train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(iris.data, iris.target, train_size=0.80)

2. Create the H2O Dataframe with column names and target column name

In [4]:
train_h2o_df = h2o.H2OFrame(train)
train_h2o_df.set_names(iris.feature_names)
train_h2o_df['species'] = h2o.H2OFrame(iris.target_names[labels_train])
train_h2o_df['species'] = train_h2o_df['species'].asfactor()

test_h2o_df = h2o.H2OFrame(test)
test_h2o_df.set_names(iris.feature_names)
test_h2o_df['species'] = h2o.H2OFrame(iris.target_names[labels_test])
test_h2o_df['species'] = test_h2o_df['species'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


### Train a H2O Model

In [5]:
iris_h2o_model = H2ORandomForestEstimator(
        model_id="iris_random_forest",
        ntrees=500,
        stopping_rounds=2,
        score_each_iteration=True,
        seed=1000000,
        balance_classes=False,
        histogram_type="AUTO")

# train 
iris_h2o_model.train(x=feature_names, y='species', training_frame=train_h2o_df)

# evaluate
iris_h2o_model.model_performance(test_h2o_df)

drf Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.027018702678145122
RMSE: 0.16437366783686833
LogLoss: 0.08742849256257935
Mean Per-Class Error: 0.025641025641025644

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,setosa,versicolor,virginica,Error,Rate
0,11.0,0.0,0.0,0.000000,0 / 11
1,0.0,12.0,1.0,0.076923,1 / 13
2,0.0,0.0,6.0,0.000000,0 / 6
3,11.0,12.0,7.0,0.033333,1 / 30



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.966667
1,2,1.000000
2,3,1.000000


## Wrapper class for H2O model

Next, we will need a wrapper class to make the H2O model behave like a scikit-learn model, which contextual-AI current supports. The wrapper mainly needs a predict function to fulfill the requirements that takes in a numpy array / pandas DataFrame and output an numpy array as prediction probability, while inside the wrapper, the input data is formatted to H2O DataFrame and feed to H2O model.

In [6]:
class h2o_predict_proba_wrapper:
    def __init__(self,model,column_names):
        """
        model: H2O model object
        column_names: feature names from the training data frame
        """
        self.model = model
        self.column_names = column_names
 
    def predict_proba(self,data_array):
        '''
        data_array: can be both numpy array type or pandas data frame
        '''
        shape_tuple = np.shape(data_array)        
        if len(shape_tuple) == 1:
            data_array = data_array.reshape(1, -1)
        
        # Convert a numpy array into H2O data frame
        pandas_df = pd.DataFrame(data = data_array,columns = self.column_names)
        h2o_df = h2o.H2OFrame(pandas_df)
        
        # Predict with the H2O model
        predictions = self.model.predict(h2o_df).as_data_frame()
        
        # the first column is the class labels, the rest are probabilities for each class
        predictions = predictions.iloc[:,1:].as_matrix()
        return predictions

### Simple test on the wrapper

In [7]:
# initialize the wrapper
h2o_drf_wrapper = h2o_predict_proba_wrapper(iris_h2o_model,feature_names) 

# convert H2O dataframe to pandas dataframe
test_pandas_df = test_h2o_df[feature_names].as_data_frame() 

# call predict_proba from the wrapper to get the prediction probability
h2o_drf_wrapper.predict_proba(test_pandas_df)


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/Users/i309943/opt/anaconda3/envs/xai/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


array([[0.94694507, 0.04983921, 0.00321572],
       [0.        , 0.99387182, 0.00612818],
       [0.        , 0.99621895, 0.00378105],
       [0.99387182, 0.        , 0.00612818],
       [0.        , 0.        , 1.        ],
       [0.        , 0.90199057, 0.09800943],
       [0.        , 0.0502451 , 0.9497549 ],
       [0.99387182, 0.        , 0.00612818],
       [0.99387182, 0.        , 0.00612818],
       [0.        , 0.0502451 , 0.9497549 ],
       [0.        , 0.99387182, 0.00612818],
       [0.99387182, 0.        , 0.00612818],
       [0.        , 0.        , 1.        ],
       [0.        , 0.99661561, 0.00338439],
       [0.        , 0.99642826, 0.00357174],
       [0.99387182, 0.        , 0.00612818],
       [0.        , 0.99661561, 0.00338439],
       [0.        , 0.99353357, 0.00646643],
       [0.99387182, 0.        , 0.00612818],
       [0.99387182, 0.        , 0.00612818],
       [0.        , 0.99387182, 0.00612818],
       [0.05230904, 0.94156278, 0.00612818],
       [0.

## Contextual-AI Report Demo

1. Define the global virables

In [8]:
# convert data into pandas dataframe
train_pandas_df = train_h2o_df[feature_names].as_data_frame() 

# initialize the h2o wrapper
h2o_drf_wrapper = h2o_predict_proba_wrapper(iris_h2o_model,feature_names) 

# define the global variables used in the config file
X_train = train_pandas_df
target_names_list = ["versicolor","virginica","setosa"]
clf = h2o_drf_wrapper
clf_fn = h2o_drf_wrapper.predict_proba
feature_names = train_pandas_df.columns.tolist()

2. Define the config json file used for compiler

In [9]:
import json
json_config = 'iris_config.json'
with open(json_config) as file:
    config = json.load(file)
config

{'name': 'Report for Iris H2O Models Feature Importance Ranking',
 'overview': True,
 'content_table': True,
 'contents': [{'title': 'Feature Importance Ranking with Iris [H2O]',
   'desc': 'This section provides the Feature Importance of model',
   'sections': [{'title': 'Feature Importance Analysis',
     'desc': 'This section provides the analysis on feature',
     'component': {'_comment': 'refer to document section xxxx',
      'class': 'FeatureImportanceRanking',
      'attr': {'trained_model': 'var:clf',
       'train_data': 'var:X_train',
       'feature_names': 'var:feature_names',
       'method': 'shap',
       'mode': 'classification'}}},
    {'title': 'Model Interpreter Analysis ',
     'desc': 'Model and train data from Housing Prices',
     'component': {'_comment': 'refer to document section xxxx',
      'class': 'ModelInterpreter',
      'attr': {'domain': 'tabular',
       'method': 'lime',
       'mode': 'classification',
       'labels': [],
       'num_of_class': 3

3. Import the corresponding module and render the report

In [11]:
sys.path.append('../../..')
from xai.compiler.base import Configuration, Controller

controller = Controller(config=Configuration(config, locals()))
controller.render()

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/Users/i309943/opt/anaconda3/envs/xai/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

/Users/i309943/opt/anaconda3/envs/xai/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


...

/Users/i309943/opt/anaconda3/envs/xai/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/Users/i309943/opt/anaconda3/envs/xai/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/Users/i309943/opt/anaconda3/envs/xai/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
../../../xai/formatter/portable_document/publisher.py:454: UserWarning: Warning: figure will exceed the page bottom, adding a new page.
  warnings.warn(message='Warning: figure will exceed the page bottom, '
